# SCIBERT in HuggingFace

In [1]:
# run this script to import the variables and settings from ipynb_util.py
%run ipynb_util.py

In [2]:
from datasets import load_dataset

# load the dataset
# note: if you don't have the data in the folder, use the download-data.sh script
dataset = load_dataset("json", data_files=str(DATA_DIR_PATH / "task1.jsonl")).class_encode_column("SDG")
dataset = dataset["train"].train_test_split(test_size=0.3, stratify_by_column="SDG", seed=SEED)

example = dataset["train"][0]
print("Example instance:\t", example)

labels = set(dataset["train"]["SDG"])
# identity because labels are already ids and vice-versa
id2label = lambda i: i
label2id = id2label
labels

Generating train split: 0 examples [00:00, ? examples/s]

Stringifying the column:   0%|          | 0/430 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/430 [00:00<?, ? examples/s]

Example instance:	 {'ID': 'oai:www.zora.uzh.ch:168503', 'TITLE': 'The carbon bubble and the pricing of bank loans', 'ABSTRACT': 'Neglecting the possibility that fossil fuel reserves can become ‘stranded’ could result in a ‘carbon bubble’ as fossil fuel firms become overvalued. This column studies whether banks price the climate policy risk of fossil fuel firms. Prior to 2015, banks did not appear to price climate policy risk. After 2015, however, the risk is priced to a certain extent, especially for firms holding more fossil fuel reserves.', 'URL': 'https://www.zora.uzh.ch/id/eprint/168503', 'SDG': 5}


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}

In [3]:
from transformers import AutoTokenizer

# base model
HF_MODEL_NAME = "allenai/scibert_scivocab_uncased"
# final model
MODEL_NAME = f"{HF_MODEL_NAME}-ft-task1"

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)

def preprocess_data(instances):
    # take a batch of titles and abstracts and concat them
    titles = instances["TITLE"]
    abstracts = instances["ABSTRACT"]
    texts = [f"{title} {abstract}" for title, abstract in zip(titles, abstracts)]
    # encode
    encoding = tokenizer(texts, padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    # add labels
    encoding["label"] = torch.tensor([label2id(label) for label in instances["SDG"]])

    return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

Map:   0%|          | 0/301 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

In [4]:
example = encoded_dataset["train"][0]
print("Example instance:\t", example)

tokenizer.decode(example["input_ids"])

Example instance:	 {'input_ids': tensor([  102,   111,  3473, 15925,   137,   111, 12775,   131,  5108, 20227,
        29116,   111,  4047,   198, 17634,  8086, 20387,   300,  3063,  1384,
        11161,   119,  5459,   968,  1186,   121,   106,  1384,  3473, 15925,
         5459,   188, 17634,  8086,  5244,  3063,   573, 11748,   205,   238,
         3724,   826,  1681,  9679,  4048,   111,  6002,  2951,  1265,   131,
        17634,  8086,  5244,   205,  1979,   147,  4505,   422,  9679,  1544,
          302,  1853,   147,  4048,  6002,  2951,  1265,   205,   647,  4505,
          422,   694,   422,   111,  1265,   165,  4048, 30118,   147,   106,
         2361,  3796,   422,  2825,   168,  5244, 11803,   475, 17634,  8086,
        20387,   205,   103,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

'[CLS] the carbon bubble and the pricing of bank loans neglecting the possibility that fossil fuel reserves can become ‘ stranded ’ could result in a ‘ carbon bubble ’ as fossil fuel firms become overvalued. this column studies whether banks price the climate policy risk of fossil fuel firms. prior to 2015, banks did not appear to price climate policy risk. after 2015, however, the risk is priced to a certain extent, especially for firms holding more fossil fuel reserves. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    HF_MODEL_NAME,
    num_labels=len(labels)
)

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 12
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/{MODEL_NAME}",
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Detected kernel version 4.19.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [7]:
trainer.train()

/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
10,2.443700,2.292044,0.364341,0.132744,0.364341,0.194591
20,2.261300,2.152836,0.364341,0.132744,0.364341,0.194591
30,1.950600,2.037476,0.387597,0.204955,0.387597,0.241803
40,1.784100,1.933329,0.465116,0.303963,0.465116,0.349424
50,1.541400,1.839436,0.503876,0.373604,0.503876,0.395588
60,1.430100,1.784764,0.488372,0.345318,0.488372,0.393404
70,1.204800,1.744094,0.503876,0.396392,0.503876,0.439102
80,1.036700,1.723695,0.496124,0.381137,0.496124,0.422098
90,0.964200,1.719974,0.503876,0.425652,0.503876,0.455959
100,0.841000,1.710862,0.488372,0.401942,0.488372,0.438292


/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/m

TrainOutput(global_step=140, training_loss=1.3060132231031145, metrics={'train_runtime': 363.4347, 'train_samples_per_second': 16.564, 'train_steps_per_second': 0.385, 'total_flos': 1584156096552960.0, 'train_loss': 1.3060132231031145, 'epoch': 20.0})

## Evaluation

In [8]:
trainer.evaluate()

/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.71113121509552,
 'eval_accuracy': 0.5193798449612403,
 'eval_precision': 0.4534535980440935,
 'eval_recall': 0.5193798449612403,
 'eval_f1': 0.4799861777625288,
 'eval_runtime': 1.2185,
 'eval_samples_per_second': 105.872,
 'eval_steps_per_second': 2.462,
 'epoch': 20.0}

## Inference

In [9]:
text = "This is a test text about climate change."

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)

In [10]:
logits = outputs.logits
logits.shape

torch.Size([1, 18])

In [11]:
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())

# for multi-label classification, we need to threshold the probabilities
"""
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
predicted_labels = [id2label(i) for i in np.where(predictions == 1)]
print(predicted_labels)
"""
# for single-label classification, we can take the argmax
predictions = probs.argmax(-1)
predicted_label = id2label(predictions)
print(predicted_label)

tensor(5)


In [12]:
import gc

def my_reset(*varnames):
    """
    Resets global variables and majority of CUDA memory. Only works in Jupyter.

    varnames are what you want to keep
    """
    try:
        del model
    except NameError:
        pass
    gc.collect()
    with torch.no_grad():
        torch.cuda.ipc_collect()
        torch.cuda.empty_cache()

    globals_ = globals()
    to_save = {v: globals_[v] for v in varnames}
    to_save['my_reset'] = my_reset  # lets keep this function by default
    del globals_
    get_ipython().magic("reset")
    globals().update(to_save)

# my_reset()

/tmp/ipykernel_27454/3741609671.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic("reset")


Nothing done.


### Investigating sklearn precision warning

In [13]:
p = torch.ones((86, 18)).argmax(-1)
p[:10] = torch.Tensor([i for i in range(10)])
t = torch.ones(86, dtype=torch.long)
t[:18] = torch.Tensor([i for i in range(18)])

# no Warning
precision_recall_fscore_support(t, p, average="weighted", labels=[i for i in range(10)])

(0.9873459873459873, 0.1282051282051282, 0.12816755893678972, None)

In [14]:
# Warning
precision_recall_fscore_support(t, p, average="weighted", labels=[i for i in range(11)])

/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.9748479368732533, 0.12658227848101267, 0.12654518477303286, None)